In [13]:
import argparse
import json
import os
import sys
import torch
import torch.nn as nn
import re
import math
from tqdm import tqdm
from transformers import (
    AutoModel,
    AutoModelForCausalLM,
    AutoTokenizer,
    CLIPVisionModelWithProjection,
    ViTForImageClassification,
)
from torch.utils.data import DataLoader
import numpy as np
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

In [14]:
import models

# device = torch.device("cpu")
device = torch.device(f"cuda:0" if torch.cuda.is_available() else "cpu")

model_path = '/home/jgryu/Weight_compression/VQVAE/checkpoint/nwc_ql/block_ql_col_16/lmbda100_rdloss_ql_encdim512_batch_size4096_total_iter1500000_lr0.0001_seed100/best_loss_model_loss_12.00164_bpp_6.0_MSE_0.02411_total_iter_450000.pth.tar'
model_path = '/home/jgryu/Weight_compression/VQVAE/checkpoint/nwc_ql_random/block_ql_random_col_128/lmbda100_rdloss_ql_encdim512_batch_size4096_total_iter1500000_lr0.0001_seed100/best_loss_model_loss_4.38201_bpp_5.515_MSE_0.00803_total_iter_1330000.pth.tar'
# model_path = '/home/jgryu/Weight_compression/VQVAE/checkpoint/nwc_ql_random/block_ql_random_col_128/lmbda1000_rdloss_ql_encdim512_batch_size4096_total_iter1500000_lr0.0001_seed100/best_loss_model_loss_6.22785_bpp_7.30125_MSE_0.00081_total_iter_1330000.pth.tar'
model_path = '/home/jgryu/Weight_compression/VQVAE/checkpoint/nwc/block_row_16/lmbda100_rdloss_encdim512_batch_size4096_total_iter1500000_lr0.0001_seed100/best_mse_model_bpp_6.394_MSE_0.00727_total_iter_1390000.pth.tar'

config = os.path.join(os.path.dirname(model_path), 'config.json')
with open(config, 'r', encoding='utf-8') as file:
    config = json.load(file)

comp_model = models.SimpleVAECompressionModel(
    input_size=config['input_size'],
    dim_encoder=config['dim_encoder'],
    n_resblock=config['n_resblock'],
    scale=torch.zeros(config['input_size']),
    shift=torch.zeros(config['input_size'])
)

ckpt = torch.load(model_path)
comp_model.load_state_dict(ckpt["state_dict"])
comp_model.to(device)
    

/tmp/ipykernel_1454676/843036172.py:23: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(model_path)


SimpleVAECompressionModel(
  (g_a): Sequential(
    (0): Linear(in_features=16, out_features=512, bias=True)
    (1): ResidualStack(
      (stack): ModuleList(
        (0-3): 4 x Linear_ResBlock(
          (lin_1): Sequential(
            (0): Linear(in_features=512, out_features=512, bias=True)
            (1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
            (2): ReLU()
          )
        )
      )
    )
    (2): Linear(in_features=512, out_features=16, bias=True)
  )
  (g_s): Sequential(
    (0): Linear(in_features=16, out_features=512, bias=True)
    (1): ResidualStack(
      (stack): ModuleList(
        (0-3): 4 x Linear_ResBlock(
          (lin_1): Sequential(
            (0): Linear(in_features=512, out_features=512, bias=True)
            (1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
            (2): ReLU()
          )
        )
      )
    )
    (2): Linear(in_features=512, out_features=16, bias=True)
  )
  (entropy_bottleneck): EntropyBottleneck

In [15]:
# comp_model.quality_embedding.weight.shape

In [16]:
# import numpy as np
# import matplotlib.pyplot as plt
# from sklearn.manifold import TSNE

# tsne = TSNE(n_components=2, perplexity=2, random_state=42)  # perplexity는 1 이상이어야 함
# vectors_embedded = tsne.fit_transform(comp_model.quality_embedding.weight[0:4].detach().numpy())

# plt.figure(figsize=(8, 6))
# for i, (x, y) in enumerate(vectors_embedded):
#     plt.scatter(x, y, label=f"Vector {i+1}")
#     # plt.text(x + 0.02, y + 0.02, f"V{i+1}", fontsize=9)  # 벡터 번호 추가

# plt.title("t-SNE Visualization of 5 Vectors")
# plt.xlabel("t-SNE Component 1")
# plt.ylabel("t-SNE Component 2")
# plt.legend()
# plt.show()


In [17]:

# empty_model = models.NWC_ql(
#     input_size=config['input_size'],
#     dim_encoder=config['dim_encoder'],
#     n_resblock=config['n_resblock'],
#     scale=torch.zeros(config['input_size']),
#     shift=torch.zeros(config['input_size'])
# )


# import numpy as np
# import matplotlib.pyplot as plt
# from sklearn.manifold import TSNE

# tsne = TSNE(n_components=2, perplexity=2, random_state=42)  # perplexity는 1 이상이어야 함
# vectors_embedded = tsne.fit_transform(empty_model.quality_embedding.weight[0:4].detach().numpy())

# plt.figure(figsize=(8, 6))
# for i, (x, y) in enumerate(vectors_embedded):
#     plt.scatter(x, y, label=f"Vector {i+1}")
#     # plt.text(x + 0.02, y + 0.02, f"V{i+1}", fontsize=9)  # 벡터 번호 추가

# plt.title("t-SNE Visualization of 5 Vectors")
# plt.xlabel("t-SNE Component 1")
# plt.ylabel("t-SNE Component 2")
# plt.legend()
# plt.show()


In [22]:
from datasets_weight_block_qlevel import get_dataset_block_qlevel, LayerInputs
from datasets_weight_block_qlevel_random import get_dataset_block_random_qlevel
from datasets_weight_block_qlevel_random import get_dataset_block_random_qlevel
from datasets_weight_block import get_datasets_block

# train_dataset, test_dataset, train_std, test_std = get_dataset_block_random_qlevel('../Wparam_dataset/dataset_block/meta-llama/Meta-Llama-3-8B/mlp_attn_128_col_dataset.pt')
# train_dataset, test_dataset, train_std, test_std = get_dataset_block_qlevel('col', 16)
train_dataset, test_dataset, train_std, test_std = get_datasets_block('../Wparam_dataset/dataset_block/meta-llama/Meta-Llama-3-8B/mlp_attn_16_row_dataset.pt')

/home/jgryu/Weight_compression/VQVAE/datasets_weight_block.py:32: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data = torch.load(dataset_folder_path)


In [26]:
def test(test_dataset, model, criterion):
    mean_MSE = 0
    avg_bpp = 0
    mean_loss = 0
    mean_recon_loss = 0
    mean_bpp_loss = 0
    device = next(model.parameters()).device
    mse_func = nn.MSELoss()

    batch_size = 100  # 원하는 배치 크기
    num_samples = len(test_dataset)

    with torch.no_grad():
        for i in range(0, num_samples, batch_size):
            batch = {key: [] for key in test_dataset[0].keys()}

            # 배치 데이터를 수집
            for idx in range(i, min(i + batch_size, num_samples)):
                data = test_dataset[idx]
                for key, value in data.items():
                    batch[key].append(value)

            # 리스트를 텐서로 변환 후 GPU로 이동
            batch = {key: torch.stack(value).to(device) for key, value in batch.items()}

            # Forward pass through the model
            out_net = model(batch)
            out_loss = criterion(data=batch, output=out_net)

            # Accumulate losses
            mean_loss += out_loss['loss'].item() * len(batch['weight_block'])
            mean_recon_loss += out_loss['recon_loss'].item() * len(batch['weight_block'])
            mean_bpp_loss += out_loss['bpp_loss'].item() * len(batch['weight_block'])

            # Compression and decompression
            
            batch['weight_block'] =  batch['weight_block'].permute(0, 2, 1).contiguous()
            
            out_enc = model.compress(batch)
            out_dec = model.decompress(out_enc["strings"], torch.size(16))

            # Calculate BPP and MSE for the batch
            num_pixels = batch['weight_block'].numel() // batch['weight_block'].size(0)  # Per image

            batch_bpp = 0
            for s in out_enc["strings"]:
                batch_bpp += sum(len(string) for string in s) * 8.0 / num_pixels

            x_hat = out_dec["x_hat"].clone().detach()
            mean_MSE += mse_func(batch['weight_block'], x_hat).item() * len(batch['weight_block'])
            avg_bpp += batch_bpp * len(batch['weight_block'])

    # Normalize accumulated metrics by the total number of samples
    mean_loss /= num_samples
    mean_recon_loss /= num_samples
    mean_bpp_loss /= num_samples
    mean_MSE /= num_samples
    avg_bpp /= num_samples

    return {
        'TEST MSE': mean_MSE,
        'TEST BPP': avg_bpp,
        'TEST loss': mean_loss,
        'TEST recon_loss': mean_recon_loss,
        'TEST bpp_loss': mean_bpp_loss
    }


In [24]:
from loss import *

criterion = RateDistortionLoss(train_std, lmbda=10)

In [25]:
comp_model = comp_model.eval().to(device)
comp_model.requires_grad_(False)
comp_model.update()

result = test(test_dataset, comp_model, criterion)

TypeError: decode_with_indexes(): incompatible function arguments. The following argument types are supported:
    1. (self: compressai.ans.RansDecoder, arg0: str, arg1: list[int], arg2: list[list[int]], arg3: list[int], arg4: list[int]) -> list[int]

Invoked with: <compressai.ans.RansDecoder object at 0x7fe231bc7670>, [b'\x8b\xbaths*\x04\x00\xebE\x13W', b'\xb0uj\xd1\xb5k\xf7\x01W=\xa4\xf9', b'\xd1\xc5\xe5\x13\xb7\xc6\xca\x00Z\xb6,\xfb', b'-\xc6?rq\xa6\x104\xdf>s\x9e', b'\xb5\xbc\x1f\x99\xdb\x0b\x19\x11a\x83\xc4\x07', b' \xc9 Q\xd7\xcc\x18\x002\xddo\x06', b'\xf7Mx\xe7\x00\x00\x00\x00P\xc8 V\x04\x03y\x12', b'\xc6Ug+%\xf4\xd3\x07?\xc3\xfe\xe6', b'B\xec\xe5\xaf\x05\x00\x00\x00\xcc\x7f\x88\xe5^\x19\xf2\xe1', b'd[V\x87\xe3M\x02\x00\xdd7\xe2X', b'~\x92\x02\xb0N\x88<\x00LT\x05\xc9', b'l\xde\x9c\x86\x10\x00\x00\x00\xcfN\x15:\x0b\x1b\xf2<', b'\x074ror\xca\t\x00\x8eHuU', b'\xd6L!\xef\xc6j\x1a\x00\x86\x8a\x16g', b"v@\xc2\x12\xcb\x867\x01'\xc0W\xfc", b"\xd1/\xce\x96\xd7\xcb'p\xfdyE\xed", b'\x0e\xc

In [7]:
indices = torch.randperm(len(train_dataset))  # 데이터셋 길이만큼 무작위 인덱스 생성

In [ ]:
datas = {}
for ql in range(4):
    data_ = []
    n = 0
    for i in indices[:1000000]:
        data = train_dataset[i]
        if data['q_level'] == ql:
            data_.append(data)
            n += 1
            if n == 500: break
    print(n)
    datas[ql] = data_

In [9]:
outs = {}
for ql in range(4):
    out_ = []
    
    for data in datas[ql]:
        data = {key: tensor.unsqueeze(0).to(device) for key, tensor in data.items()}

        out_net = comp_model(data)
        out_.append(out_net)
    outs[ql] = out_

In [10]:
mses = {}
std = 0.012528747320175171
mse_fn = nn.MSELoss()
for ql in range(4):
    mse_ = []
    for out in outs[ql]:
        mse_.append(mse_fn(out['x'], out['x_hat']).item())
    
    mses[ql] = np.array(mse_).mean() / std ** 2



In [ ]:
for ql in range(4):
    print(ql, mses[ql])

In [12]:
# 0 0.02060998404537972
# 1 0.010955656458541706
# 2 0.0018706787592914606
# 3 0.0010826996227265911

0 0.02421522901695616
1 0.01249026141716976
2 0.003785866645109464
3 0.00438829171409325

0 0.03074731674599537
1 0.020813048131938707
2 0.012604427266820322
3 0.05009694816633827

In [ ]:
y_list = []
y_hat_list = []

for idx, data in enumerate(test_dataset):
    data = {key: tensor.unsqueeze(0).to(device) for key, tensor in data.items()}
    
    out_net = comp_model(data)
    # print(out_net)
    y_list.append(out_net["y"].detach().cpu())
    y_hat_list.append(out_net["y_hat"].detach().cpu())
    
y_list = torch.cat(y_list, dim = 0)
y_hat_list = torch.cat(y_hat_list, dim = 0)
print(y_list.shape)
print(y_hat_list.shape)

In [ ]:
# tsne = TSNE(n_components=2, perplexity=100, random_state=42)  # perplexity는 1 이상이어야 함
# vectors_embedded = tsne.fit_transform(y_list)

# plt.figure(figsize=(8, 6))

# for ql in range(4):
#     plt.scatter(vectors_embedded[test_dataset['q_level'] == ql, 0], vectors_embedded[test_dataset['q_level'] == ql, 1], label=f'ql {ql}')

# plt.title("t-SNE Visualization of 5 Vectors")
# plt.xlabel("t-SNE Component 1")
# plt.ylabel("t-SNE Component 2")
# plt.legend()
# plt.show()

In [ ]:
vectors_embedded.shape

In [ ]:
print(ql_list == 1)